# Загрузка данных

In [83]:
import pandas as pd
import json

In [84]:
FILE_NAME='data10-04.xlsx'

In [85]:
df = pd.read_excel(f'datasets/{FILE_NAME}')

In [86]:
df.head()

,F_User,C_Record_ID,C_Table,N_Operation,C_Json,D_Date_Created
0,64b18dc9-69d0-4286-887f-7469ebc1b47d,9770f581-b89f-4fb3-94f5-93173bb85ad1,ED_Meter_Readings-ListView,1,"{""LINK"":""50fb47a1-6711-4e65-9232-3700daaa58bd""...",2023-04-09 16:42:47.637
1,64b18dc9-69d0-4286-887f-7469ebc1b47d,5f97a8be-b3ad-4982-9191-fbed301b3c69,DD_Doc_Details-ListView,1,"{""LINK"":""a95caaa0-a674-4f31-84d4-cbb52f0317f1""...",2023-04-09 16:42:47.980
2,64b18dc9-69d0-4286-887f-7469ebc1b47d,d4c614dd-3114-4326-8ddd-bb9d74382b03,DD_Docs_Out-ListView,0,"{""LINK"":""b1c13924-3f99-4e6c-a02a-34472f2c5320""...",2023-04-09 16:42:48.627
3,64b18dc9-69d0-4286-887f-7469ebc1b47d,9ee02cde-4a6f-4c51-8e3b-b14ddae38bee,DD_Files-ListView,0,"{""LINK"":""e010d6d0-1df5-790d-19fe-b007d7be4207""...",2023-04-09 16:42:49.557
4,64b18dc9-69d0-4286-887f-7469ebc1b47d,6829a2cd-007c-45d8-88ca-e143617555c4,DD_Docs-ListView,1,"{""LINK"":""b1c13924-3f99-4e6c-a02a-34472f2c5320""...",2023-04-09 16:42:50.010


In [87]:
df['D_Date_Created'] = pd.to_datetime(df['D_Date_Created'])
df['N_Operation'] = df['N_Operation'].astype('int8')

In [88]:
df.drop_duplicates(inplace=True)

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29561 entries, 0 to 31944
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   F_User          29561 non-null  object        
 1   C_Record_ID     29561 non-null  object        
 2   C_Table         29561 non-null  object        
 3   N_Operation     29561 non-null  int8          
 4   C_Json          29216 non-null  object        
 5   D_Date_Created  29561 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int8(1), object(4)
memory usage: 1.4+ MB


# Алгоритмы

In [90]:
users = df['F_User'].unique()

In [91]:
use_records = []

## Алгоритм 1

Последовательность DD_Doc_Details-ListView -> DD_Docs_Out-ListView -> DD_Docs-ListView (Отмена заявки на ограничение)

In [92]:
dict_algoritm_1 = {}
for i in range(len(users)):
    dict_algoritm_1[users[i]] = []

In [93]:
def algoritm_1(user_id):    
    items = df.loc[df['F_User'] == user_id].sort_values(by=['D_Date_Created'], ascending=True)

    indexes = list(items.index)
    tables = list(items['C_Table'])
    operations = list(items['N_Operation'])
    json_items = list(items['C_Json'])
    resords = list(items['C_Record_ID'])

    buffer = []
    
    for i in range(len(tables)):
        table = tables[i]
        operation = operations[i]
        json_item = json_items[i]
        index = indexes[i]
        record = resords[i]

        if table == 'DD_Doc_Details-ListView':
            if len(buffer) == 3:
                if buffer[1][4]['LINK'] == buffer[2][4]['LINK'] and buffer[0][4]['F_Docs']['LINK'] == buffer[2][4]['LINK']:
                    for j in range(len(buffer)):
                        # обновляем docs_out
                        buffer[1][4]['LINK'] = buffer[1][1]
                        buffer[1][4]['F_Doc_Details']['LINK'] = buffer[0][4]['LINK']
                        
                        dict_algoritm_1[user_id].append(buffer[j])
                        use_records.append(buffer[j][1])
                
            buffer = []
            
            buffer.append([index, record, table, operation, json.loads(json_item)])
        
        if table == 'DD_Docs_Out-ListView' and len(buffer) == 1:
            buffer.append([index, record, table, operation, json.loads(json_item)])
            
        if table == 'DD_Docs-ListView' and len(buffer) == 2:
            buffer.append([index, record, table, operation, json.loads(json_item)])
            
    # дозаписываем данные
    if len(buffer) == 3:
        for j in range(len(buffer)):
            dict_algoritm_1[user_id].append(buffer[j])

In [94]:
for i in range(len(users)):
    user_id = users[i]
    algoritm_1(user_id)

In [95]:
df = df[~df['C_Record_ID'].isin(use_records)]

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18809 entries, 0 to 31944
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   F_User          18809 non-null  object        
 1   C_Record_ID     18809 non-null  object        
 2   C_Table         18809 non-null  object        
 3   N_Operation     18809 non-null  int8          
 4   C_Json          18464 non-null  object        
 5   D_Date_Created  18809 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int8(1), object(4)
memory usage: 900.0+ KB
